<!--NOTEBOOK_HEADER-->
*This notebook contains material from [PyRosetta](https://RosettaCommons.github.io/PyRosetta.notebooks);
content is available [on Github](https://github.com/RosettaCommons/PyRosetta.notebooks.git).*

<!--NAVIGATION-->
< [RosettaCarbohydrates: Trees, Selectors and Movers](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/13.01-Glycan-Trees-Selectors-and-Movers.ipynb) | [Contents](toc.ipynb) | [Index](index.ipynb) | [RNA in PyRosetta](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/14.00-RNA-Basics.ipynb) ><p><a href="https://colab.research.google.com/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/13.02-Glycan-Modeling-and-Design.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>

# RosettaCarbohydrates: Modeling and Design
Keywords: carbohydrate, glycan, glucose, mannose, sugar, design, prediction

## Overview
Here, you will learn how to model glycans and design optimal glycosylation positions in a protein.

We will be using the RosettaCarbohydrate framework to build and model glycans.  The `GlycanModeler`, which is our main method for modeling glycans, will be published in 2020.  We will be using some custom glycan options to load pdbs. 
First, one needs the `-include_sugars` option, which will tell Rosetta to load sugars and add the sugar_bb energy term to a default scorefunction.  This scoreterm is like rama for the sugar dihedrals which connect each sugar residue. 

		-include_sugars


When loading structures from the PDB that include glycans, we use these options. This includes an option to write out the structures in pdb format instead of the Rosetta format (which is actually better).  Again, this is included in the config/flags files you will be using.

		-maintain_links
		-auto_detect_glycan_connections
		-alternate_3_letter_codes pdb_sugar
		-write_glycan_pdb_codes


More information on working with glycans can be found at this page: [Working With Glycans](https://www.rosettacommons.org/docs/latest/application_documentation/carbohydrates/WorkingWithGlycans)

## Algorithm
  
The `GlycanModeler` essentially builds glycans from the root (The first residue of the Tree) out to the trees in a way that simulates a tree growing.  It uses a notion of a 'layer' where the layer is defined as the number of residues to the glycan root (with the glycan root being layer 0).  Within modeling, all glycan residues other than the ones being optimized are 'virtualized'.  In Rosetta, the term 'Virtual' means that these residues are present, but not scored.  (It should be noted that it is now possible to turn any residues Virtual and back to Real using two movers: `ConvertVirtualToRealMover` and `ConvertRealToVirtualMover`. )

Within the modeling application, sampling of glycan DOFs is done through the `GlycanSampler`.  The sampler attempts to sample the large amount of DOFs available to a glycan tree.  The GlycanSampler is a `WeightedRandomSampler`, which is a container of highly specific sampling strategies, where each strategy is weighted by a particular probability.  At each apply, the mover selects one of these samplers using the probability set to it. This is the same way the SnugDock algorithm for antibody modeling works. 

Sampling is always scaled with the number of glycan residues that you are modeling, so run-time will increase proportionally as well. 
If you are modeling a huge viral particle with lots of glycans, one can use quench mode, which will optimize each glycan individually. 
Tpyically for these cases, multiple rounds of glycan modeling is desired. 


### GlycanSampler Major components

Some of these components were covered in the previous tutorial.

1. __Glycan Conformers__

	These conformers have been generated through an in-depth bioinformatic analysis of the PDB using adaptive kernal density estimates and are unique for each linkage type including glycan residues connected to ASN residues.  A conformer is a specific conformation of all of the backbone dihedrals of a particular glycan linkage. Essentialy glycan 'fragments' for a particular type of linkage.


2. __SugarBB Sampling__ 

	This sampling is done through turning the `sugar_bb` energy term into a set of probabilities using the -log(e) function.  This allows us to sample on the QM derived torsonal potentials during modeling. 


3. __Random Sampling and Shear Moves__

	We sample random torsions at +/- 15 , +/- 45, +/- 90 degrees, each at decreasing probabilities at a 4:2:1 ratio of sampling Small,Medium,Large. 
	Shear sampling is done where torsions are set for two residues in order to reduce downsteam effects and allow 'flipping' of the glycan torsions.


4. __Minimization__
	
	We Minimize Sugar residues by randomly selecting a residue from what is set to model, and selecting all residues out to the tree that are not virtualized. This reduces computational time that would otherwise restrict the total number of glycan residues we could model at once.
    

5. __Packing__

	Of the residues set to optimize, we chooses a random residue and pack that residue and all residues out to the tree that are not virtualized. We pack the sugar residues (OH and constituents) and any neighboring protein sidechains. TaskOperations may be set to allow design of protein residues during this.  We do packing this way to once again reduce total computational time.



In [ ]:
# Notebook setup
import sys
if 'google.colab' in sys.modules:
    !pip install pyrosettacolabsetup
    import pyrosettacolabsetup
    pyrosettacolabsetup.setup()
    print ("Notebook is set for PyRosetta use in Colab.  Have fun!")

**Make sure you are in the directory with the pdb files:**

`cd google_drive/My\ Drive/student-notebooks/`

# General Setup and Inputs

You will be using a few different inputs.  We will be designing in glycosylation spots in order to block antibody binding at a highly curved epitope, and we will be loading a human structure from the PDB that has internal glycans.   


## Notes for Tutorial Shortening


Typically, the value of `-glycan_sampler_rounds` is set to 25 (which typically is enough) and nstruct is about 5-10k per input structure. You may increase glycan_sampler_rounds to 100 and then decrease output to 1-2500 nstruct in order to have the same level of sampling, which will result in very good models as well.  Since this is denovo modeling of glycans, more nstruct is almost always better. For some tutorials, we may decrease this value below our optimal value in order to shorten the length of the tutorial.


## General Notes

We will use a flags file for all common options in this tutorial.  Note that instead of passing this flag on init, you can instead put it into your working directory or a particular place in your home directory and rename it common. 
    
See this page for more info on using rosetta with custom config files: <https://www.rosettacommons.org/docs/latest/rosetta_basics/running-rosetta-with-options#common-options-and-default-user-configuration>

All tutorials have generated output in output_files and their approximate time to finish on a single (core i7) processor.


In [3]:
#Python
from pyrosetta import *
from pyrosetta.rosetta import *
from pyrosetta.teaching import *
init('@inputs/glycans/common_glycans @inputs/glycans/pdb_flags @inputs/glycans/map_flags')

PyRosetta-4 2019 [Rosetta PyRosetta4.Release.python36.mac 2019.39+release.93456a567a8125cafdf7f8cb44400bc20b570d81 2019-09-26T14:24:44] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Reading fconfig.../Users/jadolfbr/.rosetta/flags/common
core.init: 
core.init: 
core.init: Rosetta version: PyRosetta4.Release.python36.mac r233 2019.39+release.93456a567a8 93456a567a8125cafdf7f8cb44400bc20b570d81 http://www.pyrosetta.org 2019-09-26T14:24:44
core.init: command: PyRosetta @inputs/glycans/common_glycans @inputs/glycans/pdb_flags @inputs/glycans/map_flags -database /Users/jadolfbr/Library/Python/3.6/lib/python/site-packages/pyrosetta-2019.39+release.93456a567a8-py3.6-macosx-10.6-intel.egg/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=2011870376 seed_offset=0 real_

# Tutorial

GlycanModeling is done through the RosettaScripts interface.  Each tutorial has you copying a base XML and adding/modifying specific components to achieve a goal.  ALL of these movers are available as components in PyRosetta - however, setup is much more difficult and time consuming.  So for now, we will rely on the RS interface, but 

## Tutorial A: Epitope Blocking, De-novo Glycan Modeling

Here, we will start with the antigen known as Bee Hyaluronidase, from PDB ID 2J88.  The PDB file has an antibody bound to it as a HIGHLY immunogenic site. We would like to block this in order to begin to use this enzyme for therapy as Hyaluronidase can be effective in breaking down sugars in the extracellular matrix, allowing certain larger drugs to get to regions of interest.  The antibody is renumbered into the AHo numbering scheme that we used in the RAbD tutorial, and it has been relaxed with constraints into the Rosetta energy function. 

We will be designing in at least one optimal glycan at the most immunogenic site.
Note that a prototocol called SugarCoat is in development that will scan regions of interest for potential ideal glycosylation, however, one can certainly do this manually as we do below. 

### A 1. Designing in a Glycosylation Site: 

CreateGlycanSequonMover` and `CreateSequenceMotifMover`

A sugar glycosylation site is known as a `Sequon`.  The glycan sequon is made up of three protein residues which are recognized by the GlycosylTransferase Enzyme during translation in the ER.  This enzyme adds the root of nascent glycan onto a protein.  In this case, we use the sequon for ASN glycosylation.  The sequon is as follows: `N[^P][S/T]`.  The `[^P]` notation means that any residue other than P can be there.  The `[S/T]` notation means that either S or T is recognized.  This notation can be used to directly create Motifs in proteins using the `CreateSequenceMotifMover` and associated `SequenceMotifTaskOperation`. Documentation for these is available here:

<https://www.rosettacommons.org/docs/wiki/scripting_documentation/RosettaScripts/xsd/mover_CreateSequenceMotifMover_type>
	<https://www.rosettacommons.org/docs/wiki/scripting_documentation/RosettaScripts/xsd/to_SequenceMotifTaskOperation_type>

The create GlycanSequonMover can also be used for glycosylation of different AA than ASN.

#### A1.1 Design using a typical sequon

Before we begin, take a look at the complex either by PyMol or use the PyMolMover as you have previously. 
The file is `inputs/glycans/2j88_complex.pdb`
        
Where can we introduce a glycan to block binding?
        
        
Where do you think the optimal glycan position would be for this particular antibody?  Take a look at the xml.  Is this the position we are targeting?  Typically, we may want to allow some backbone movement in our sequon.  The full glycan scanning protocol can be found in an input file, simple_glycan_scanner_manual.xml, where we relax the motif residues with constraints, add the sequon, and then relax again, comparing the energy between them to get the full energetic contributions of the sequon on the structure.  In order to reduce the run time in these tutorials, we will be removing this going forward.

The XML syntax is below: 
            
     <CreateGlycanSequeonMover name="motif_creator" residue_selector="select"/>
            
Go ahead and run the xml (`inputs/glycans/tutA11.xml`) using what you have learned previously, or run the code below (about 15 seconds)

Note that the xml uses `SimpleMetrics` to output a variety of metrics that are in pose.scores or output into a scorefile.

Here is the full XML.  We will only use part of it in code.

```xml
<ROSETTASCRIPTS>
    <SCOREFXNS>
    </SCOREFXNS>
    <RESIDUE_SELECTORS>
        <Index name="select" resnums="%%start%%" />
        <Index name="motif" resnums="%%start%%-%%end%%" />
        <Neighborhood name="nbrhood" selector="motif"/>
        <Not name="others" selector="nbrhood" />
    </RESIDUE_SELECTORS>
    <SIMPLE_METRICS>
        <TimingProfileMetric name="timing"/>
        <TotalEnergyMetric name="total_energy_delta" use_native="1" custom_type="native_delta"/>
        <TotalEnergyMetric name="total_energy" use_native="0"/>
        <CompositeEnergyMetric name="composite_energy" use_native="1"/>
        <SelectedResiduesMetric name="selection" residue_selector="select"/>
		<SelectedResiduesMetric name="rosetta_sele" residue_selector="select" rosetta_numbering="1"/>
		<SelectedResiduesPyMOLMetric name="pymol_selection" residue_selector="select" />
        <SelectedResiduesPyMOLMetric name="region" residue_selector="nbrhood" />
        <SequenceMetric name="sequence" residue_selector="motif" />
        <SasaMetric name="sasa" residue_selector="select" />
        <RMSDMetric name="rmsd" use_native="1" rmsd_type="rmsd_protein_bb_heavy"/>
    </SIMPLE_METRICS>
    <MOVERS>
        <CreateGlycanSequonMover name="motif_creator" residue_selector="select" pack_neighbors="1"/>
        <RunSimpleMetrics name="selections" metrics="rosetta_sele,pymol_selection,sequence" />
        <RunSimpleMetrics name="pre_metrics" metrics="sasa,total_energy" prefix="pre_" />
        <RunSimpleMetrics name="post_sequon_metrics" metrics="total_energy_delta,sequence,sasa,timing,rmsd" prefix="post-sequon_" />
    </MOVERS>
    <PROTOCOLS>
        <Add mover="selections" />
        <Add mover="pre_metrics" />
        <Add mover="motif_creator"/>
        <Add mover="post_sequon_metrics"/>
    </PROTOCOLS>
</ROSETTASCRIPTS>

```

In [39]:
from rosetta.protocols.carbohydrates import *
from rosetta.core.select.residue_selector import *
from rosetta.core.simple_metrics.metrics import *
from rosetta.core.simple_metrics.composite_metrics import *
from rosetta.core.simple_metrics.per_residue_metrics import *

In [38]:
pose_complex = pose_from_pdb("inputs/glycans/2j88_complex.pdb")
pose_complex_original = pose_complex.clone()

pose_antigen = pose_from_pdb("inputs/glycans/2j88_antigen.pdb")
pose_antigen_original = pose_antigen.clone()

core.import_pose.import_pose: File 'inputs/glycans/2j88_complex.pdb' automatically determined to be of type PDB
core.io.pdb.pdb_reader: Parsing 660 .pdb records with unknown format to search for Rosetta-specific comments.
core.io.util: Automatic glycan connection is activated.
core.io.util: Start reordering residues.
core.io.util: Corrected glycan residue order (internal numbering): []
core.io.util: 
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] missing density found at residue (rosetta number) 56
core.conformation.Residue: [ WARNING ] Residue connection id changed when creating a new residue at seqpos 57
core.conformation.Residue: [ WARNING ] ResConnID info stored on the connected residue (residue 0) is now out of date!
core.conformation.Residue: [ WARNING ] Connection atom name (in src):  C
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] missing density found at residue (rosetta number) 359
core.conformation.Residue: [ WARNING ] Residue connection id changed when creati

In [47]:
start = pose_antigen.pdb_info().pdb2pose("A", 143)
end = pose_antigen.pdb_info().pdb2pose("A", 145)
select = ResidueIndexSelector()
select.set_index(start)

motif = ResidueIndexSelector()
motif.set_index_range(start, end)

sequon_creator = CreateGlycanSequonMover(select)
sequon_creator.apply(pose_antigen)

protocols.carbohydrates.CreateGlycanSequonMover: Sequons created
protocols.carbohydrates.CreateGlycanSequonMover: Applying...
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.scoring.ScoreFunctionFactory: The -include_sugars flag was used with no sugar_bb weight set in the weights file.  Setting sugar_bb weight to 1.0 by default.
basic.io.database: Database file opened: scoring/score_functions/elec_cp_reps.dat
core.scoring.elec.util: Read 40 countpair representative atoms
core.pack.dunbrack.RotamerLibrary: shapovalov_lib_fixes_enable option is true.
core.pack.dunbrack.RotamerLibrary: shapovalov_lib::shap_dun10_smooth_level of 1( aka lowest_smooth ) got activated.
core.pack.dunbrack.RotamerLibrary: Binary rotamer library selected: /Users/jadolfbr/Library/Python/3.6/lib/python/site-packages/pyrosetta-2019.39+release.93456a567a8-py3.6-macosx-10.6-intel.egg/pyrosetta/database/rotamer/shapovalov/StpDwn_0-0-0/Dunbrack10.lib.bin
core.pack.dunbrack.RotamerLibrary: Using Dunbrack 

In [53]:
seq_metric = SequenceMetric(motif)
print("original", seq_metric.calculate(pose_antigen_original))
print("designed", seq_metric.calculate(pose_antigen))

original FWD
designed NWS


Did we design in the sequon?  (`N[^P][S/T]`)

What do our scores look like?
Note we are packing neighbors here, so our score does not totally say that we actually reduced the score.

In [54]:
score = get_score_function()
print("original", score.score(pose_antigen_original))
print("designed", score.score(pose_antigen))

core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.scoring.ScoreFunctionFactory: The -include_sugars flag was used with no sugar_bb weight set in the weights file.  Setting sugar_bb weight to 1.0 by default.
original -643.5306907221014
designed -654.908075182865


Note we are packing neighbors here, so our score does not totally say that we actually reduced the score.

#### A1.2 Design using the `N[^P][T]` motif


This motif has been shown to have higher occupancy of the glycosation site with glycans in the resulting protein.  Glycosylation is not 100% in some cases at some positions for (currently) unknown reasons, but it has been shown that a THR at the +2 site results in higher occupancy. If we were creating a drug, we can use chromatography during protein isolation to choose peaks which include our glycan. 

Here, we are using the [-] notation as to not actually design the second position.  We will use what is in the native protein here. Again, you may run the XML (`inputs/glycans/tutA12.xml`) within PyRosetta replacing start and end with: `start=143A end=145A`, or run the code below. 

The key XML line is here:
    <CreateSequenceMotifMover name="create_sequon" residue_selector="p1" motif="N[-]T"/>


In [56]:
from rosetta.protocols.calc_taskop_movers import *

general_motif_creator = CreateSequenceMotifMover(select)
general_motif_creator.set_motif("N[-]T")
pose_antigen_d1 = pose_antigen.clone()

general_motif_creator.apply(pose_antigen)

core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.scoring.ScoreFunctionFactory: The -include_sugars flag was used with no sugar_bb weight set in the weights file.  Setting sugar_bb weight to 1.0 by default.
protocols.calc_taskop_movers.CreateSequenceMotifMover: Focus [129, 130, 131]Applying...
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 108 rotamers at 9 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating PDInteractionGraph


Was the sequon successfully designed?

In [57]:
print("original", seq_metric.calculate(pose_antigen_original))
print("designed1", seq_metric.calculate(pose_antigen_d1))
print("designed2", seq_metric.calculate(pose_antigen))

original FWD
designed1 NWS
designed NWT


In [58]:
print("original", score.score(pose_antigen_original))
print("designed1", score.score(pose_antigen_d1))
print("designed2", score.score(pose_antigen))  

original -643.5306907221014
designed1 -654.9080751828645
designed2 -653.9549454739722


### A2. Adding a man5 glycan: 

__SimpleGlycosylateMover__

Now, we will expand on our first tutorial by glycosylating afterward. We will use the common name for a man5 sugar, which is a high-mannose branching glycan of 7 sugar residues (and 5 mannoses).  You can use a few common names to make glycosylation easier, or an IUPAC string, or a file that has the IUPAC string in the first name of the file. Common names include man5,man7,man9 and a few others.  You can find these in

	database/chemical/carbohydrates/common_glycans

The IUPAC nomenclature of the man5 is as follows:  


		a-D-Manp-(1->3)-[a-D-Manp-(1->3)-[a-D-Manp-(1->6)]-a-D-Manp-(1->6)]
		                                          -b-D-Manp-(1->4)-b-D-GlcpNAc-(1->4)-b-D-GlcpNAc-



More information on IUPAC nomenclature of sugar trees is here: <http://www.chem.qmul.ac.uk/iupac/2carb>.  There is also a very detailed README in the common glycan directory for your reference.

Note that within the `SimpleGlycosylateMover` you may also give multiple glycans using the `glycans` option, which will randomly choose a glycan tree to use for glycosylation from the list given.  Glycosylation is not deterministic in that you always get a man5 at a particular position and is influenced by a great deal of structural biology that is not yet fully determined.  For now, since we are aiming to create a drug and purifying our result, using a man5 is sufficient. Once again, either run the xml (inputs/glycans/tutA2.xml) as before or run the code below. This takes about 15 seconds. 

XML syntax is as follows:

	<SimpleGlycosylateMover name="glycosylate" residue_selector="select" glycan="man5" />



In [59]:
glycosylate = SimpleGlycosylateMover()
glycosylate.set_glycosylation("man5")
glycosylate.set_residue_selector(select)
glycosylate.apply(pose_antigen)

core.chemical.carbohydrates.database_io: N-glycan_core N-glycan_core.iupac
core.chemical.carbohydrates.database_io: glycan_core N-glycan_core.iupac
core.chemical.carbohydrates.database_io: core N-glycan_core.iupac
core.chemical.carbohydrates.database_io: man3 N-glycan_core.iupac
core.chemical.carbohydrates.database_io: man5 man5.iupac
core.chemical.carbohydrates.database_io: man9 man9.iupac
core.chemical.carbohydrates.database_io: bisected_N-glycan_core bisected_N-glycan_core.iupac
core.chemical.carbohydrates.database_io: bisected_fucosylated_N-glycan_core bisected_fucosylated_N-glycan_core.iupac
core.chemical.carbohydrates.database_io: core_1_O-glycan core_1_O-glycan.iupac
core.chemical.carbohydrates.database_io: core_2_O-glycan core_2_O-glycan.iupac
core.chemical.carbohydrates.database_io: core_3_O-glycan core_3_O-glycan.iupac
core.chemical.carbohydrates.database_io: core_4_O-glycan core_4_O-glycan.iupac
core.chemical.carbohydrates.database_io: core_5_O-glycan core_5_O-glycan.iupac
c

Take a look at the structure using the PyMol Mover or output it.  What does the structure look like? The SimpleGlycosylateMover simply adds in glycans at ideal values for rings and backbones. 
Lets optimize this and model some glycans!

### A3. Modeling glycans

`GlycanResidueSelector` and the `GlycanModeler`

We will run the previous tutorials in a single rosetta script where we end with modeling the glycan residues.  We use a very short run time and nstruct, so results will not be as clean as they would otherwise, but this should give you an idea of how all this works. Typically, we would model different positions of potential glycosylations, but here to save time, we will simply continue to build and model the glycan position we started with. Output files have been provided for you if you wish to use these. We will not be giving the mover a residue selector as it uses all glycans by default, but you can use the `GlycanResidueSelector` to choose specific trees or even glycan residues within those trees to model as we have seen in the previous tutorial.

This takes about 380 seconds to run.  You can either use the XML (`inputs/glycans/tutA3.xml`) or the code below. Output 10 structures.

XML Syntax:

	<GlycanModeler name="model" layer_size="2" window_size="1" rounds="1" refine="false" />


In [61]:
#Here, we setup some defaults that will be set in master shortly. 
# Note that the name of the GTM will change to simply GlycanModeler within the next week or two.

modeler = GlycanTreeModeler()
modeler.set_hybrid_protocol(True)
modeler.set_use_shear(True)
modeler.set_use_gaussian_sampling(True)
modeler.apply(pose_antigen)

core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.scoring.ScoreFunctionFactory: The -include_sugars flag was used with no sugar_bb weight set in the weights file.  Setting sugar_bb weight to 1.0 by default.
protocols.carbohydrates.GlycanSampler: Randomizing glycan torsions
protocols.carbohydrates.GlycanTreeModeler: Smallest glycan layer: 0
protocols.carbohydrates.GlycanTreeModeler: Largest  glycan layer: 4
protocols.carbohydrates.GlycanTreeModeler: Ntrees to model 1
protocols.carbohydrates.GlycanTreeModeler: Starting Score: 7529.89
protocols.carbohydrates.GlycanTreeModeler: starting round 1
protocols.carbohydrates.GlycanTreeModeler: Going in the forward direction
protocols.carbohydrates.GlycanTreeModeler: Modeling up to max end: 4
protocols.carbohydrates.GlycanTreeModeler: Virtualizing new foliage layer
protocols.carbohydrates.GlycanTreeModeler: Running the GlycanSampler on layer [ start -> end (including) ]: 0 1
protocols.carbohydrates.GlycanTreeModeler: Applying normal p

protocols.carbohydrates.GlycanSampler: initializing objects
protocols.carbohydrates.GlycanSampler: Modeling 2 glycan residues
protocols.carbohydrates.GlycanSampler: Initialized
protocols.carbohydrates.GlycanSampler: starting energy: -469.11
protocols.carbohydrates.GlycanSampler: Total Rounds = 24 ( 2 glycans * 12 )
protocols.carbohydrates.LinkageConformerMover: Sampling ->4)-beta-D-GlcpNAc-(?->4)-beta-D-GlcpNAc- linkage
protocols.carbohydrates.LinkageConformerMover: Sampling conformer 2 which has a population of 0.0124
protocols.carbohydrates.LinkageConformerMover: Complete
protocols.carbohydrates.LinkageConformerMover: Sampling ->4)-beta-D-GlcpNAc-(?->4)-beta-D-GlcpNAc- linkage
protocols.carbohydrates.LinkageConformerMover: Sampling conformer 2 which has a population of 0.0124
protocols.carbohydrates.LinkageConformerMover: Complete
protocols.carbohydrates.LinkageConformerMover: Sampling ->4)-beta-D-GlcpNAc-(?->4)-beta-D-GlcpNAc- linkage
protocols.carbohydrates.LinkageConformerMover: S

core.pack.pack_rotamers: built 1078 rotamers at 7 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
core.pack.task: Packer task: initialize from command line()
core.pack.rotamer_set.RotamerSet_: [ WARNING ] including current in order to get at least 1 rotamer !!!!!! 129 ASN:N-glycosylated
core.pack.pack_rotamers: built 1079 rotamers at 8 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
protocols.carbohydrates.GlycanTreeMinMover: Minimizing from carbohydrate root: 321
protocols.carbohydrates.GlycanTreeMinMover: Minimizing 2 residues
core.conformation.Conformation: [ WARNING ] Branch 1does not have enough heavy atoms about its connection to define a torsion angle!
core.conformation.Conformation: [ WARNING ] Branch 1does not have enough heavy atoms about its connection to define a torsion angle!
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: bu

protocols.carbohydrates.util: min accept: 1
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 427 rotamers at 4 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
protocols.carbohydrates.util: pack accepted: 1
core.conformation.Conformation: [ WARNING ] Branch 1does not have enough heavy atoms about its connection to define a torsion angle!
protocols.carbohydrates.GlycanSampler: energy final post min: -631.385
protocols.carbohydrates.GlycanSampler: energy recov post min: -632.623
protocols.moves.TrialCounter:              unk trials=    269;  accepts= 0.8625;  energy_drop/trial=  -8.74065
protocols.carbohydrates.GlycanTreeModeler: Running full window protocol sampling
protocols.carbohydrates.GlycanSampler: initializing objects
protocols.carbohydrates.GlycanSampler: Modeling 5 glycan residues
protocols.carbohydrates.GlycanSampler: Initialized
protocols.carbohydrates.GlycanSampler: starting ener

core.pack.rotamer_set.RotamerSet_: [ WARNING ] including current in order to get at least 1 rotamer !!!!!! 129 ASN:N-glycosylated
core.pack.pack_rotamers: built 591 rotamers at 6 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
protocols.carbohydrates.util: pack accepted: 1
core.conformation.Conformation: [ WARNING ] Branch 1does not have enough heavy atoms about its connection to define a torsion angle!
protocols.carbohydrates.GlycanSampler: energy final post min: -633.67
protocols.carbohydrates.GlycanSampler: energy recov post min: -634.869
protocols.moves.TrialCounter:              unk trials=    441;  accepts= 0.8912;  energy_drop/trial=  -0.00232
protocols.carbohydrates.GlycanTreeModeler: De-Virtualizing current foliage layer
protocols.carbohydrates.GlycanTreeModeler: Virtualizing new foliage layer
protocols.carbohydrates.GlycanTreeModeler: Running the GlycanSampler on layer [ start -> end (including) ]: 3 4
protocols.carbohyd

...

protocols.carbohydrates.GlycanTreeMinMover: Minimizing 1 residues
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 1078 rotamers at 7 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
protocols.carbohydrates.GlycanSampler: Round: 20
core.pack.task: Packer task: initialize from command line()
core.pack.rotamer_set.RotamerSet_: [ WARNING ] including current in order to get at least 1 rotamer !!!!!! 129 ASN:N-glycosylated
core.pack.pack_rotamers: built 1079 rotamers at 8 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
protocols.carbohydrates.GlycanTreeMinMover: Minimizing from carbohydrate root: 322
protocols.carbohydrates.GlycanTreeMinMover: Minimizing 5 residues
protocols.carbohydrates.LinkageConformerMover: Sampling alpha-D-Manp-(?->3)-beta-D-Manp- linkage
protocols.carbohydrates.LinkageConformerMover: Sampling conformer 1 which has a p

protocols.carbohydrates.LinkageConformerMover: Complete
protocols.carbohydrates.LinkageConformerMover: Sampling alpha-D-Manp-(?->6)-alpha-D-Manp- linkage
protocols.carbohydrates.LinkageConformerMover: Sampling conformer 2 which has a population of 0.4401
protocols.carbohydrates.LinkageConformerMover: Complete
protocols.carbohydrates.LinkageConformerMover: Sampling alpha-D-Manp-(?->3)-alpha-D-Manp- linkage
protocols.carbohydrates.LinkageConformerMover: Sampling conformer 1 which has a population of 1
protocols.carbohydrates.LinkageConformerMover: Complete
protocols.carbohydrates.LinkageConformerMover: Sampling alpha-D-Manp-(?->3)-alpha-D-Manp- linkage
protocols.carbohydrates.LinkageConformerMover: Sampling conformer 1 which has a population of 1
protocols.carbohydrates.LinkageConformerMover: Complete
protocols.simple_moves.BackboneMover: [ WARNING ] no movable positions in ShearMover!
protocols.carbohydrates.GlycanSampler: Last mover failed.  Continueing!
protocols.carbohydrates.GlycanS

protocols.simple_moves.BackboneMover: [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: [ WARNING ] no m

protocols.simple_moves.BackboneMover: [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: [ WARNING ] no movable positions in ShearMover!
protocols.simple_moves.BackboneMover: [ WARNING ] no m

core.pack.task: Packer task: initialize from command line()
core.pack.rotamer_set.RotamerSet_: [ WARNING ] including current in order to get at least 1 rotamer !!!!!! 129 ASN:N-glycosylated
core.pack.pack_rotamers: built 1079 rotamers at 8 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
protocols.carbohydrates.GlycanSampler: Round: 20
protocols.carbohydrates.LinkageConformerMover: Sampling ->4)-beta-D-GlcpNAc-(?ASN linkage
protocols.carbohydrates.LinkageConformerMover: Sampling conformer 6 which has a population of 0.0063
protocols.carbohydrates.LinkageConformerMover: Complete
protocols.carbohydrates.LinkageConformerMover: Sampling alpha-D-Manp-(?->3)-alpha-D-Manp- linkage
protocols.carbohydrates.LinkageConformerMover: Sampling conformer 1 which has a population of 1
protocols.carbohydrates.LinkageConformerMover: Complete
protocols.carbohydrates.LinkageConformerMover: Sampling alpha-D-Manp-(?->6)-alpha-D-Manp- linkage
protocols.ca

Run this code for a total of 10 nstruct.  Compare energies and take a look at the lowest one. Scores and pdbs are also available in `expected_outputs/glycans` with prefix `tutA3`

How does it look?  Load the native (complexed structure) into pymol as well.  Would this glycan block this particular antibody? Where else could we place a glycan?

## Tutorial B: Using Glycan Density

In this tutorial we will load a pdb directly into Rosetta with sugars already present. The config for this has been provided for you.  We will exclusively be using the XML interface to PyRosetta for these tutorials, however, using the knowledge gained in the Density and Symmetry tutorials, all of this is available code-wise as well, but the XML interface greatly simplifies this task.


The glycan tree that we will be working with is 5 residues long.  I use coot to look at density maps. 
Density maps were generated by downloading the cif file from the PDB and using PHENIX maps and default `maps.params`. This command was used to generate them.  This is covered in the Working-With-Density tutorial:

		phenix.maps inputs/glycans/4do4.pdb inputs/glycans/4do4-sf.cif 

The density map generated is too large to be distributed with the rest of the tuorial, so I have uploaded it to Google Drive for you to download. <https://drive.google.com/open?id=1h569jpwLxyHu7iHLG8eu2Q9_B-Q9e_C9> Please download and place it in your working directory, if it's not already there.

The tutorial will use a density map that is small enough to fit into the repo, but you are welcome to use 4d04 instead with the downloaded map.


### B1. Calculating Density Fit


Although a structure may be solved with high resolution, not all solved residues may fit the density well.  A structure from the PDB is still a model afterall, informed through experimentation.  This is especially true of glycan residues, which are fairly mobile.  Crystal contacts of neighboring proteins help to reduce the movemment of glycans and may help to induce a state that can be solved more easily given high-resolution density.  In this tutorial, we will be using Rosetta to determine how well a residue fits into the given density.  There are methods to do this in the coot program, but we want to be able to do this for any structure in a streamlined way - especially if we need to calculate RMSDs on only well-fitting glycan residues.  The methods we will be employing in Rosetta are based on Frank Dimaio's work with Rosetta density.  

To do this, we will once again be employing the SimpleMetric system.  In this case, we use the `PerResidueDensityFitMetric`, which is a PerResidueRealMetric.  This type of SimpleMetric calculates a particular value for each residue given a residue selector.  Very useful here. 

We will also be employing the DensityFitResidueSelector, which uses the metric.  Since this is a fairly slow metric, we will use in-built functionality for using our calculated values from the metric, which are stored in the pose.  We will then use the SelectedResidueCountMetric to determine how many residues have great fit.  In later tutorials, we will be using the RMSDMetric with this selector in order to calculate RMSD on well-fitting glycan residues. 

	Residues higher than .8 are great fit to density.
	Residues between .6 - .8 are good fit to density
	Residues below .4 fit to density are BAD fits


The XML is `inputs/glycans/tutB1.xml`

The key XML lines are:


	<PerResidueDensityFitMetric name="fit_native" residue_selector="tree" output_as_pdb_nums="1" 
			                                                     sliding_window_size="1" match_res="1"/>
	<DensityFitResidueSelector name="fits8" den_fit_metric="fit_native" cutoff=".8" use_cache="1" 
			                                                                  fail_on_missing_cache="1"/>
	<SelectedResidueCountMetric name="n_fits8" custom_type="fit8" residue_selector="fits8"/>

Run the xml and while it is running, take a look at the XML (runtime is about 80 seconds).  It is fairly complicated and we will be building on it during the rest of these tutorials.  Note that we first define the density metric, and then we use it within the selector.  At the bottom, we add these to our set of native_metrics.  What other metrics are we using?


In [4]:
branch="200A"
map_file="inputs/1jnd_2mFo-DFc_map.ccp4"
symmdef="inputs/1jnd_crys.symm"

xml = f'''
<ROSETTASCRIPTS>
    <SCOREFXNS>
    </SCOREFXNS>

    NEEDED FOR CACHING density fit info
    <RESIDUE_SELECTORS>
        <Glycan name="tree" branch="{branch}" include_root="0" />
    </RESIDUE_SELECTORS>
    <SIMPLE_METRICS>
        <PerResidueDensityFitMetric name="fit_native" residue_selector="tree" output_as_pdb_nums="1" sliding_window_size="1" match_res="1"/>
    </SIMPLE_METRICS>
    END Density Fit Setup

    <RESIDUE_SELECTORS>
        
        <Index name="root" resnums="{branch}" />
        <GlycanLayerSelector name="first_layer" start="0" end="1"/>
        <And name="layer01" selectors="tree,first_layer" />
        <DensityFitResidueSelector name="fits8" den_fit_metric="fit_native" cutoff=".8" use_cache="1" fail_on_missing_cache="1" prefix="native_"/>
        <DensityFitResidueSelector name="fits6" den_fit_metric="fit_native" cutoff=".6" use_cache="1" fail_on_missing_cache="1" prefix="native_"/>
        <DensityFitResidueSelector name="fits4" den_fit_metric="fit_native" cutoff=".4" use_cache="1" fail_on_missing_cache="1" prefix="native_"/>
    </RESIDUE_SELECTORS>

    <SIMPLE_METRICS>

        <TimingProfileMetric name="timing" />

        <SelectedResidueCountMetric name="n_tree" custom_type="tree_size" residue_selector="tree"/>
        <SelectedResidueCountMetric name="n_fits8" custom_type="fit8" residue_selector="fits8"/>
        <SelectedResidueCountMetric name="n_fits6" custom_type="fit6" residue_selector="fits6"/>
        <SelectedResidueCountMetric name="n_fits4" custom_type="fit4" residue_selector="fits4"/>
        <SelectedResidueCountMetric name="n_layer01" custom_type="layer01" residue_selector="layer01"/>

        <PerResidueGlycanLayerMetric   name="layers"         residue_selector="tree" output_as_pdb_nums="1"/>
        <SelectedResiduesPyMOLMetric   name="pymol_tree"   residue_selector="tree"   custom_type="glycans"/>
        <SelectedResiduesPyMOLMetric   name="pymol_branch" residue_selector="root"   custom_type="branch"/>
        <SelectedResiduesMetric        name="pdb_glycans"  residue_selector="tree"   rosetta_numbering="0" custom_type="glycans"/>
        <SelectedResiduesMetric        name="pdb_branch"   residue_selector="root"   rosetta_numbering="0" custom_type="branch"/>

    </SIMPLE_METRICS>
    <MOVERS>
        <SetupForSymmetry name="setup_symm" definition="{symmdef}"/>
        <LoadDensityMap name="loaddens" mapfile="{map_file}"/>
        <SetupForDensityScoring name="setupdens"/>
        <RunSimpleMetrics name="native_metrics" metrics="fit_native" prefix="native_" />
        <RunSimpleMetrics name="selections" metrics="layers,pymol_tree,pymol_branch,pdb_glycans,pdb_branch" />
        <RunSimpleMetrics name="counts"     metrics="n_tree,n_layer01,n_fits6,n_fits8"/>
        <RunSimpleMetrics name="timings"    metrics="timing" /> 

    </MOVERS>
    <PROTOCOLS>
        <Add mover_name="loaddens"/>
        <Add mover_name="setupdens"/>
        <Add mover_name="selections"/>
        <Add mover_name="native_metrics" />
        <Add mover_name="counts"/>
        <Add mover_name="timings"/>
    </PROTOCOLS>
</ROSETTASCRIPTS>
'''

In [5]:
pose = pose_from_pdb("inputs/1jnd_refined.pdb.gz")

core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 1251 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 1.1842 seconds.
core.import_pose.import_pose: File 'inputs/1jnd_refined.pdb.gz' automatically determined to be of type PDB
core.io.pdb.pdb_reader: Parsing 411 .pdb records with unknown format to search for Rosetta-specific comments.
core.io.util: Automatic glycan connection is activated.
core.io.util: Start reordering residues.
core.io.util: Corrected glycan residue order (internal numbering): [401, 402, 403, 404]
core.io.util: 
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] missing density found at residue (rosetta number) 140
core.conformation.Residue: [ WARNING ] Residue connection id changed when creating a new residue at seqpos 141
core.conformation.Residue: [ WARNING ] ResConnID info stored on the connected residue (residue 0) is now out of date!
core.conformation.Residue: [ WARNING ] Connection ato

In [6]:
mover = pyrosetta.rosetta.protocols.rosetta_scripts.XmlObjects.create_from_string(xml).get_mover("ParsedProtocol")

protocols.rosetta_scripts.RosettaScriptsParser: Generating XML Schema for rosetta_scripts...
protocols.rosetta_scripts.RosettaScriptsParser: ...done
protocols.rosetta_scripts.RosettaScriptsParser: Initializing schema validator...
protocols.rosetta_scripts.RosettaScriptsParser: ...done
protocols.rosetta_scripts.RosettaScriptsParser: Validating input script...
protocols.rosetta_scripts.RosettaScriptsParser: ...done
protocols.rosetta_scripts.RosettaScriptsParser: Parsed script:
<ROSETTASCRIPTS>
	<SCOREFXNS/>
	<RESIDUE_SELECTORS>
		<Glycan branch="200A" include_root="0" name="tree"/>
	</RESIDUE_SELECTORS>
	<SIMPLE_METRICS>
		<PerResidueDensityFitMetric match_res="1" name="fit_native" output_as_pdb_nums="1" residue_selector="tree" sliding_window_size="1"/>
	</SIMPLE_METRICS>
	<RESIDUE_SELECTORS>
		<Index name="root" resnums="200A"/>
		<GlycanLayerSelector end="1" name="first_layer" start="0"/>
		<And name="layer01" selectors="tree,first_layer"/>
		<DensityFitResidueSelector cutoff=".8" den_

core.conformation.symmetry.SymmData: [ WARNING ] Setting jump_group JUMPGROUP2: [master 10]  20:0  22:0  12:0  14:0  16:0  18:0  24:0  26:0
core.conformation.symmetry.SymmData: symmetry name: 1jnd_P_32_2_1
core.conformation.symmetry.SymmData: number of subunits: 9
core.conformation.symmetry.SymmData: number of interfaces: 1
core.conformation.symmetry.SymmData: score subunit number: VRT_0_0_0_0
core.conformation.symmetry.SymmData: anchor the subunits at residue: 1
core.conformation.symmetry.SymmData:  Virtual coordinate system VRT_0_0_0_0
core.conformation.symmetry.SymmData: x: 1 0 0
core.conformation.symmetry.SymmData: y: 0 1 0
core.conformation.symmetry.SymmData: origin: 73.8756 49.0221 11.025
core.conformation.symmetry.SymmData:  Virtual coordinate system VRT_0_0_0_0_base
core.conformation.symmetry.SymmData: x: 1 0 0
core.conformation.symmetry.SymmData: y: 0 1 0
core.conformation.symmetry.SymmData: origin: 72.8756 48.0221 10.025
core.conformation.symmetry.SymmData:  Virtual coordinat

core.conformation.symmetry.SymmData: Jump JUMP_4_2_1_n1 VRT_0_0_0_0 VRT_4_2_1_n1
core.conformation.symmetry.SymmData: Jump JUMP_4_2_1_n1_to_com VRT_4_2_1_n1 VRT_4_2_1_n1_base
core.conformation.symmetry.SymmData: Jump JUMP_4_2_1_n1_to_subunit VRT_4_2_1_n1_base SUBUNIT
core.conformation.symmetry.SymmData: Jump JUMP_5_0_1_0 VRT_0_0_0_0 VRT_5_0_1_0
core.conformation.symmetry.SymmData: Jump JUMP_5_0_1_0_to_com VRT_5_0_1_0 VRT_5_0_1_0_base
core.conformation.symmetry.SymmData: Jump JUMP_5_0_1_0_to_subunit VRT_5_0_1_0_base SUBUNIT
core.conformation.symmetry.SymmData: Jump JUMP_5_1_1_0 VRT_0_0_0_0 VRT_5_1_1_0
core.conformation.symmetry.SymmData: Jump JUMP_5_1_1_0_to_com VRT_5_1_1_0 VRT_5_1_1_0_base
core.conformation.symmetry.SymmData: Jump JUMP_5_1_1_0_to_subunit VRT_5_1_1_0_base SUBUNIT
core.conformation.symmetry.SymmData: Include subunit:
core.conformation.symmetry.SymmData: Output subunit:
core.conformation.symmetry.SymmData: SlideType: RANDOM
core.conformation.symmetry.SymmData: SlideCriter

In [7]:
mover.apply(pose)

protocols.rosetta_scripts.ParsedProtocol: =======================BEGIN MOVER LoadDensityMap - loaddens=======================
core.scoring.electron_density.ElectronDensity: Loading Density Map
core.scoring.electron_density.ElectronDensity: Loading density mapinputs/1jnd_2mFo-DFc_map.ccp4
core.scoring.electron_density.ElectronDensity:  Setting resolution to AUTO
core.scoring.electron_density.ElectronDensity:           atom mask to 3.2A
core.scoring.electron_density.ElectronDensity:             CA mask to 6A
core.scoring.electron_density.ElectronDensity:  Read density map'inputs/1jnd_2mFo-DFc_map.ccp4'
core.scoring.electron_density.ElectronDensity:      extent: 218 x 229 x 265
core.scoring.electron_density.ElectronDensity:      origin: -66 x 82 x 209
core.scoring.electron_density.ElectronDensity:   altorigin: 0 x 0 x 0
core.scoring.electron_density.ElectronDensity:        grid: 360 x 360 x 288
core.scoring.electron_density.ElectronDensity:     celldim: 106.345 x 106.345 x 89.987
core.sco

Ok, take a look at the (tutB1) scorefile or pose.scores.  You can use the scorefile.py script to output as tabs if you would like. How many residues have great fit to density (hint, look for `fit6_selection_count` and `fit8_selection_count` data terms)?  Are there any residues that fit poorly into the density?

In [8]:
print(pose.scores)

{'branch_pymol_selection': 'select rosetta_sele, (chain A and resid 200)', 'branch_selection': '200A', 'glycans_pymol_selection': 'select rosetta_sele, (chain A and resid 3100,3101,3102,3103)', 'glycans_selection': '3100A,3101A,3102A,3103A', 'fit6_selection_count': 4.0, 'fit8_selection_count': 2.0, 'glycan_layer_3100A': 0.0, 'glycan_layer_3101A': 1.0, 'glycan_layer_3102A': 2.0, 'glycan_layer_3103A': 3.0, 'layer01_selection_count': 2.0, 'native_res_density_fit_3100A': 0.850679730589326, 'native_res_density_fit_3101A': 0.8347641236660035, 'native_res_density_fit_3102A': 0.7610287167541959, 'native_res_density_fit_3103A': 0.6496957888062153, 'timing_profile': 1.0479055643081665, 'tree_size_selection_count': 4.0}


In [9]:
print(pose.scores["fit6_selection_count"])

4.0


In [10]:
print(pose.scores["fit8_selection_count"])

2.0


In [11]:
import re


for term in pose.scores:
    if re.search("native_res_density_fit", term):
        print(term.split("_")[-1], pose.scores[term])


3100A 0.850679730589326
3101A 0.8347641236660035
3102A 0.7610287167541959
3103A 0.6496957888062153


### B2. Refinement into density
		
Here, we will be doing a short refinement protocol into the density, with its crystal symmetry.  This is a short protocol, but will work for our purposes.  For a much longer (albeit very similar) refinement protocol of the glycan and whole protein, see Frenz et al (referenced in the Working With Glycans tutorial). The full protocol used in this paper is included in the input files as cryoem_glycan_refinement.xml.  Take a look and see how it compares to what we are doing here. As usual, output files are available. Runtime for all 10 structures is about 2 hours. 

The XML is `inputs/glycans/tutB2.xml`.  Since this takes so long to run, we will skip running it here. 


Take a look at the scorefile either using the scorefile.py script or through pandas: `expected_outputs/glycans/tutB2_score.sc`

Are the density fit scores higher?  How different are the RMSDs of the glycan residues? Take a look at the structure of the lowest energy - how different does it look?  Are any new contacts created?  Were we able to improve the density fit for some of those residues?

### B3. Denovo building into Density
	
In this tutorial, we will be once again loading our crystal structure with density and symmetry.  However, we will be randomizing the bb torsions and building the glycan out from scratch. In reality, we would have some idea of what glycan we are building and we would glycosylate the protein with the chemical motif we have figured out from means such as mass spec.  We would then model the glycan to solve the crystal structure. With the new PackerPalette machinery in Rosetta and the ability to design glycans, we could actually build a protocol to sample chemical motifs of the glycans we are building out into the density, however, since this a very very large combinatorial problem, we should have some idea of what exists in the structure. 


We will first rebuild the glycan tree using the density as a guide, and then refine it further using what we learned in the previous tutorial. Note that like tutorial B2, this one takes a good long while (4 hours)

The XML for this is `inputs/glycans/tutB3.xml`.  Once again, we will not be running this here.


Take a look at the scorefile and PDBs. (prefix tutB3_)

How are our RMSDs?  Were we able to do enough sampling to get close to the native structure?  Are the energies acceptable?  Are there parts of the glycan that are closer to native than others?  Why might this be?  Is an nstruct of 10 enough??

Thank you for doing this tutorial!  I hope you learned a lot and are ready to work with these crazy carbohydrates!  Cheers!

<!--NAVIGATION-->
< [RosettaCarbohydrates: Trees, Selectors and Movers](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/13.01-Glycan-Trees-Selectors-and-Movers.ipynb) | [Contents](toc.ipynb) | [Index](index.ipynb) | [RNA in PyRosetta](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/14.00-RNA-Basics.ipynb) ><p><a href="https://colab.research.google.com/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/13.02-Glycan-Modeling-and-Design.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>